In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import joblib
import logging

# ------------------------------------------------------------------------
# LOGGING & CONFIGURATION
# ------------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()]
)
LOGGER = logging.getLogger(__name__)

CONFIG = {
    "csv_path": "/kaggle/input/new-city-base-data/city_base_weather.csv",  # Update if needed
    "output_dir": "forecasts",
    "model_dir": "models",
    "initial": "365 days",
    "period": "180 days",
    "horizon": "90 days",
    "max_lag": 3,
    "prophet_param_grid": {
        "changepoint_prior_scale": [0.05, 0.1, 0.2],
        "seasonality_mode": ["additive", "multiplicative"]
    },
    "rf_param_dist": {
        "n_estimators": [50, 100, 200],
        "max_depth": [3, 5, 7, 10, None],
        "min_samples_split": [2, 5, 10],
        "max_features": ["sqrt", "log2", None]
    },
    "n_iter_rf_random_search": 10,
    "cv_splits": 3,
    "random_state": 42,
    # Used in the training loop below
    "target_year": 2026  
}

os.makedirs(CONFIG["model_dir"], exist_ok=True)
os.makedirs(CONFIG["output_dir"], exist_ok=True)

# ------------------------------------------------------------------------
# DATA LOADING FUNCTION
# ------------------------------------------------------------------------
def load_data_multi_city(csv_path: str) -> pd.DataFrame:
    LOGGER.info(f"Loading data from {csv_path} ...")
    
    if not os.path.isfile(csv_path):
        raise FileNotFoundError(f"Could not find file: {csv_path}")
    
    df = pd.read_csv(csv_path)
    
    # Check for required columns
    required_cols = ['city_name', 'date', 'rain_sum (mm)']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"CSV missing columns: {missing_cols}")

    # Rename columns to match Prophet expectations
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df.rename(columns={'date': 'ds', 'rain_sum (mm)': 'y'}, inplace=True)
    df.sort_values('ds', inplace=True)

    # Drop rows with missing ds or y
    null_mask = df['ds'].isna() | df['y'].isna()
    if null_mask.any():
        LOGGER.warning(f"Dropping {null_mask.sum()} rows with NaNs in ds or y.")
        df = df[~null_mask]

    # Clip extreme outliers (optional, helps robust training)
    df['y'] = df['y'].clip(lower=0, upper=df['y'].quantile(0.999))
    
    LOGGER.info(f"Data loaded. Shape: {df.shape}, Unique cities: {df['city_name'].nunique()}")
    return df

# ------------------------------------------------------------------------
# MODEL FUNCTIONS
# ------------------------------------------------------------------------
def tune_prophet_hyperparams(df: pd.DataFrame, param_grid: dict, initial: str, period: str, horizon: str) -> Prophet:
    """
    Tries each combination of changepoint_prior_scale and seasonality_mode. 
    Uses cross_validation to compute RMSE. If cross_validation fails (likely 
    because the data is too short), it logs a warning and assigns a large RMSE 
    to that combination. Returns the model (and hyperparams) with the lowest RMSE.
    """
    LOGGER.info("Tuning Prophet hyperparameters ...")
    best_model = None
    best_rmse = float('inf')
    
    for cps in param_grid["changepoint_prior_scale"]:
        for mode in param_grid["seasonality_mode"]:
            temp_model = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                seasonality_mode=mode,
                changepoint_prior_scale=cps
            )
            
            # Fit on entire data (Prophet requires at least 2 data points)
            temp_model.fit(df[['ds', 'y']])
            
            # Attempt cross validation
            try:
                df_cv = cross_validation(temp_model, initial=initial, period=period, horizon=horizon)
                metrics = performance_metrics(df_cv)
                rmse = metrics['rmse'].mean()
            except Exception as e:
                LOGGER.warning(
                    f"[Prophet CV] Failed for CPS={cps}, Mode={mode}, error: {str(e)}. "
                    "Assigning large RMSE=9999999 to this combination."
                )
                rmse = 9999999
            
            LOGGER.info(f"Hyperparams => CPS={cps}, Mode={mode}, Mean RMSE={rmse:.3f}")
            if rmse < best_rmse:
                best_rmse = rmse
                best_model = temp_model
    
    if best_model is None:
        # Fallback: in case *all* combos fail, just fit a default with no CV
        LOGGER.warning("All cross validation attempts failed; using default Prophet model.")
        best_model = Prophet().fit(df[['ds', 'y']])
    
    LOGGER.info(f"Best Prophet hyperparams => RMSE={best_rmse:.3f}")
    return best_model

def prophet_cross_val_evaluation(model, initial, period, horizon):
    """
    A separate cross validation step if you want to log performance metrics.
    If cross validation fails, it will log a warning and return empty DataFrames.
    """
    LOGGER.info("Running Prophet cross-validation ...")
    try:
        df_cv = cross_validation(model, initial=initial, period=period, horizon=horizon)
        df_metrics = performance_metrics(df_cv)
        LOGGER.info(f"Prophet CV metrics:\n{df_metrics.head()}")
        return df_cv, df_metrics
    except Exception as e:
        LOGGER.warning(f"Prophet cross-validation failed: {str(e)}")
        return pd.DataFrame(), pd.DataFrame()

def build_residual_dataset(df: pd.DataFrame, prophet_model: Prophet) -> pd.DataFrame:
    LOGGER.info("Creating in-sample forecast to compute residuals ...")
    in_sample_future = prophet_model.make_future_dataframe(periods=0)
    forecast = prophet_model.predict(in_sample_future)
    df_merged = pd.merge(df, forecast[['ds', 'yhat']], on='ds', how='left')
    df_merged['residual'] = df_merged['y'] - df_merged['yhat']
    return df_merged

def create_lag_features(df_res: pd.DataFrame, target_col='residual', max_lag=3) -> pd.DataFrame:
    LOGGER.info("Creating lag features for the residual ...")
    df_lag = df_res.copy()
    df_lag['day_of_year'] = df_lag['ds'].dt.dayofyear
    df_lag['day_of_week'] = df_lag['ds'].dt.dayofweek
    for lag in range(1, max_lag + 1):
        df_lag[f'{target_col}_lag_{lag}'] = df_lag[target_col].shift(lag)
    df_lag.dropna(inplace=True)
    return df_lag

def evaluate_model_predictions(y_true, y_pred, model_name="Model"):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    # Avoid division by zero in MAPE if y_true has zeros
    if np.all(y_true != 0):
        mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    else:
        mape = np.nan
    r2 = r2_score(y_true, y_pred)
    LOGGER.info(f"{model_name} => MAE: {mae:.3f}, RMSE: {rmse:.3f}, MAPE: {mape:.2f}%, R^2: {r2:.3f}")
    return {"mae": mae, "rmse": rmse, "mape": mape, "r2": r2}

def tune_residual_model(df_supervised: pd.DataFrame, param_dist: dict, n_iter: int, cv_splits: int, random_state: int, target_col='residual'):
    LOGGER.info("Training RandomForest on residual with RandomizedSearchCV ...")
    feature_cols = [c for c in df_supervised.columns if 'lag_' in c or 'day_of_' in c]
    X = df_supervised[feature_cols]
    y = df_supervised[target_col]
    
    tscv = TimeSeriesSplit(n_splits=cv_splits)
    rf = RandomForestRegressor(random_state=random_state)
    randomized_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=tscv,
        scoring='neg_mean_squared_error',
        random_state=random_state,
        n_jobs=-1
    )
    randomized_search.fit(X, y)
    
    best_rf_model = randomized_search.best_estimator_
    best_params = randomized_search.best_params_
    best_score = -randomized_search.best_score_
    LOGGER.info(f"Best RF Params => {best_params}, MSE={best_score:.2f}")
    
    y_pred = best_rf_model.predict(X)
    _ = evaluate_model_predictions(y, y_pred, model_name="RandomForest (Residual)")
    
    return best_rf_model, feature_cols

def iterative_residual_prediction(model, df_history, future_dates, feature_cols, max_lag=3) -> pd.DataFrame:
    """
    Step forward in time, using the newly predicted residual as a lag for the next day.
    """
    df_temp = df_history.copy()
    predictions = []
    for dt in future_dates:
        last_row = df_temp.iloc[-1].copy()
        for lag in range(1, max_lag + 1):
            if lag == 1:
                new_lag_val = last_row['residual']
            else:
                new_lag_val = last_row[f'residual_lag_{lag - 1}']
            last_row[f'residual_lag_{lag}'] = new_lag_val
        
        last_row['ds'] = dt
        last_row['day_of_year'] = dt.day_of_year
        last_row['day_of_week'] = dt.day_of_week
        
        X_new = pd.DataFrame([last_row[feature_cols]], columns=feature_cols)
        resid_pred = model.predict(X_new)[0]
        
        new_row = last_row.copy()
        new_row['ds'] = dt
        new_row['residual'] = resid_pred
        predictions.append([dt, resid_pred])
        
        df_temp = pd.concat([df_temp, pd.DataFrame([new_row])], ignore_index=True)
    
    df_pred = pd.DataFrame(predictions, columns=['ds', 'residual_pred'])
    return df_pred

def create_hybrid_forecast(prophet_model, rf_model, df_supervised, feature_cols, target_year, max_lag):
    LOGGER.info(f"Creating hybrid forecast for {target_year} ...")
    start_date = f"{target_year}-01-01"
    end_date = f"{target_year}-12-31"
    last_training_date = df_supervised['ds'].max()
    
    days_needed = (pd.to_datetime(end_date) - last_training_date).days
    if days_needed < 1:
        raise ValueError(
            f"Requested year {target_year} is before/equal to "
            f"the last training date {last_training_date}."
        )
    
    # Prophet forecast for the required days
    future_df = prophet_model.make_future_dataframe(periods=days_needed)
    prophet_forecast = prophet_model.predict(future_df)
    prophet_future = prophet_forecast[
        (prophet_forecast['ds'] >= start_date) & (prophet_forecast['ds'] <= end_date)
    ].copy()
    
    # Generate residual predictions iteratively
    # Start from the last supervised (lag) row to propagate the lags properly
    df_history_for_residual = df_supervised.iloc[[-1]].copy()
    future_dates = pd.to_datetime(prophet_future['ds'].unique())
    
    df_future_resid = iterative_residual_prediction(
        model=rf_model,
        df_history=df_history_for_residual,
        future_dates=future_dates,
        feature_cols=feature_cols,
        max_lag=max_lag
    )
    hybrid_df = prophet_future.merge(df_future_resid, on='ds', how='left')
    
    # If no residual_pred for some dates, fill them with 0
    hybrid_df['residual_pred'].fillna(0, inplace=True)
    
    # Final hybrid forecast
    hybrid_df['yhat_hybrid'] = hybrid_df['yhat'] + hybrid_df['residual_pred']
    return hybrid_df

# ------------------------------------------------------------------------
# PLOTTING FUNCTION (2-SUBPLOT STYLE)
# ------------------------------------------------------------------------
def plot_full_forecast_and_save(city: str, forecast_df: pd.DataFrame, target_year: int):
    """
    Plots the two‐subplot forecast chart and saves it as a PNG:
      - Top subplot: Observed history (black), Prophet forecast (blue w/ uncertainty), Hybrid forecast (red).
      - Bottom subplot: Monthly average of the Hybrid forecast (bar chart).
    """
    LOGGER.info(f"Plotting forecast for city={city}, year={target_year}")
    df_all = load_data_multi_city(CONFIG["csv_path"])
    df_city = df_all[df_all["city_name"] == city].copy()
    if df_city.empty:
        LOGGER.warning(f"No historical data found for city: {city}")
        return

    # Ensure ds is datetime
    if not pd.api.types.is_datetime64_any_dtype(forecast_df['ds']):
        forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])
    
    # Filter for the target year
    forecast_year = forecast_df[forecast_df['ds'].dt.year == target_year].copy()
    if forecast_year.empty:
        LOGGER.warning(f"No forecast data found for year {target_year} in {city}.")
        return

    # Observed data up to the forecast start
    forecast_start = forecast_year['ds'].min()
    observed = df_city[df_city['ds'] < forecast_start]

    # Create the figure with 2 subplots
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))

    # --- Top subplot ---
    ax1 = axes[0]
    ax1.set_title(f"Hybrid Rainfall Forecast for {city} ({target_year})")

    # Plot observed (black)
    ax1.plot(observed['ds'], observed['y'], color='black', label='Observed (History)')

    # Uncertainty band if columns exist
    if 'yhat_lower' in forecast_year.columns and 'yhat_upper' in forecast_year.columns:
        ax1.fill_between(
            forecast_year['ds'],
            forecast_year['yhat_lower'],
            forecast_year['yhat_upper'],
            color='gray',
            alpha=0.2,
            label='Prophet Uncertainty'
        )

    # Prophet Forecast (blue dashed)
    if 'yhat' in forecast_year.columns:
        ax1.plot(
            forecast_year['ds'], 
            forecast_year['yhat'], 
            linestyle='--',
            color='blue', 
            label='Prophet Baseline'
        )

    # Hybrid Forecast (red)
    if 'yhat_hybrid' in forecast_year.columns:
        ax1.plot(
            forecast_year['ds'], 
            forecast_year['yhat_hybrid'], 
            color='red', 
            label='Hybrid Forecast'
        )

    ax1.set_ylabel("Rainfall (mm)")
    ax1.legend(loc='best')

    # --- Bottom subplot ---
    ax2 = axes[1]
    ax2.set_title(f"Monthly Average Hybrid Forecast ({target_year})")

    forecast_year['month'] = forecast_year['ds'].dt.month
    monthly_avg = (
        forecast_year.groupby('month')['yhat_hybrid']
        .mean()
        .reset_index()
        .rename(columns={'yhat_hybrid': 'avg_forecast'})
    )

    ax2.bar(monthly_avg['month'], monthly_avg['avg_forecast'])
    ax2.set_xlabel("Month")
    ax2.set_ylabel("Avg Rainfall (mm)")
    ax2.set_xticks(range(1, 13))
    ax2.set_xticklabels([f"{m:02d}" for m in range(1, 13)])
    ax2.set_xlim([0.5, 12.5])

    plt.tight_layout()

    # Save the figure
    plot_filename = os.path.join(CONFIG["output_dir"], f"{city}_hybrid_forecast_{target_year}.png")
    plt.savefig(plot_filename, dpi=150)
    LOGGER.info(f"Chart saved to {plot_filename}")

    plt.show()

# ------------------------------------------------------------------------
# TRAINING LOOP FOR ALL CITIES (Runs Sequentially)
# ------------------------------------------------------------------------
df_all = load_data_multi_city(CONFIG["csv_path"])
unique_cities = df_all['city_name'].unique()
LOGGER.info(f"Found {len(unique_cities)} cities: {unique_cities}")

for city in unique_cities:
    df_city = df_all[df_all["city_name"] == city].copy()
    LOGGER.info(f"\n===== Processing city: {city} | Data shape: {df_city.shape} =====")
    
    # If the city has fewer than 2 rows, skip it (Prophet cannot train on 1 row)
    if df_city.shape[0] < 2:
        LOGGER.warning(f"Not enough data for city {city}. Skipping.")
        continue

    # 1) Train Prophet model with hyperparameter tuning
    prophet_model = tune_prophet_hyperparams(
        df=df_city,
        param_grid=CONFIG["prophet_param_grid"],
        initial=CONFIG["initial"],
        period=CONFIG["period"],
        horizon=CONFIG["horizon"]
    )
    # Attempt a cross val evaluation (separate step, but if it fails, no big deal)
    _, _ = prophet_cross_val_evaluation(
        prophet_model,
        CONFIG["initial"],
        CONFIG["period"],
        CONFIG["horizon"]
    )

    # 2) Build residual dataset and create lag features
    df_residual = build_residual_dataset(df_city, prophet_model)
    df_supervised = create_lag_features(
        df_residual, 
        target_col='residual',
        max_lag=CONFIG["max_lag"]
    )

    if df_supervised.empty:
        LOGGER.warning(f"Residual dataset for {city} is empty after lagging. Skipping city.")
        continue

    # 3) Train RandomForest on residuals
    rf_model, feature_cols = tune_residual_model(
        df_supervised=df_supervised,
        param_dist=CONFIG["rf_param_dist"],
        n_iter=CONFIG["n_iter_rf_random_search"],
        cv_splits=CONFIG["cv_splits"],
        random_state=CONFIG["random_state"]
    )

    # 4) Create the hybrid forecast for the default target_year
    target_year = CONFIG["target_year"]
    try:
        hybrid_forecast = create_hybrid_forecast(
            prophet_model=prophet_model,
            rf_model=rf_model,
            df_supervised=df_supervised,
            feature_cols=feature_cols,
            target_year=target_year,
            max_lag=CONFIG["max_lag"]
        )
    except ValueError as e:
        LOGGER.warning(f"Could not create forecast for city={city}, year={target_year}: {str(e)}")
        continue

    # 5) Plot the final two‐subplot chart for this city and year, then save it
    plot_full_forecast_and_save(city, hybrid_forecast, target_year)

    # 6) Save forecast and models
    forecast_filename = os.path.join(
        CONFIG["output_dir"],
        f"{city}_rainfall_forecast_{target_year}_hybrid.csv"
    )
    hybrid_forecast.to_csv(forecast_filename, index=False)

    prophet_model_path = os.path.join(CONFIG["model_dir"], f"{city}_prophet_model.pkl")
    rf_model_path = os.path.join(CONFIG["model_dir"], f"{city}_rf_residual_model.pkl")

    joblib.dump(prophet_model, prophet_model_path)
    joblib.dump(rf_model, rf_model_path)

    LOGGER.info(f"Forecast saved to {forecast_filename}")
    LOGGER.info(f"Prophet model saved to {prophet_model_path}")
    LOGGER.info(f"RF model saved to {rf_model_path}")

# ------------------------------------------------------------------------
# PREDICTION FUNCTION FOR A GIVEN CITY AND YEAR
# ------------------------------------------------------------------------
def predict_forecast(city: str, target_year: int) -> pd.DataFrame:
    """
    Predicts the hybrid forecast for the given city and target_year.
    It loads the saved models for the city, recomputes the residual dataset
    and lag features from the historical data, then returns the forecast.
    """
    df_all = load_data_multi_city(CONFIG["csv_path"])
    df_city = df_all[df_all["city_name"] == city].copy()
    if df_city.shape[0] < 2:
        raise ValueError(f"Not enough data for city '{city}'.")

    prophet_model_path = os.path.join(CONFIG["model_dir"], f"{city}_prophet_model.pkl")
    rf_model_path = os.path.join(CONFIG["model_dir"], f"{city}_rf_residual_model.pkl")
    if not os.path.exists(prophet_model_path) or not os.path.exists(rf_model_path):
        raise FileNotFoundError(f"Saved models for city '{city}' not found.")
    
    prophet_model = joblib.load(prophet_model_path)
    rf_model = joblib.load(rf_model_path)
    
    # Build residual dataset and create lag features
    df_residual = build_residual_dataset(df_city, prophet_model)
    df_supervised = create_lag_features(df_residual, target_col='residual', max_lag=CONFIG["max_lag"])
    if df_supervised.empty:
        raise ValueError(f"No valid residual data for city '{city}' after lagging.")
    
    feature_cols = [c for c in df_supervised.columns if 'lag_' in c or 'day_of_' in c]
    
    # Create hybrid forecast
    forecast = create_hybrid_forecast(prophet_model=prophet_model,
                                      rf_model=rf_model,
                                      df_supervised=df_supervised,
                                      feature_cols=feature_cols,
                                      target_year=target_year,
                                      max_lag=CONFIG["max_lag"])
    return forecast

def plot_full_forecast(city: str, forecast_df: pd.DataFrame, target_year: int):
    """
    Plots a full forecast chart:
      - Top subplot: observed history with Prophet forecast (with uncertainty band)
        and Hybrid forecast.
      - Bottom subplot: Monthly average of the Hybrid forecast.
      
    The function loads historical data for the city to plot the observed values.
    """
    # Load historical data for the city
    df_all = load_data_multi_city(CONFIG["csv_path"])
    df_city = df_all[df_all["city_name"] == city].copy()
    if df_city.empty:
        print(f"No historical data found for city {city}.")
        return

    # Ensure the forecast dataframe's ds column is datetime
    if not pd.api.types.is_datetime64_any_dtype(forecast_df['ds']):
        forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])
    
    # Filter forecast for the target year
    forecast_year = forecast_df[forecast_df['ds'].dt.year == target_year].copy()
    if forecast_year.empty:
        print(f"No forecast data found for the year {target_year}.")
        return

    # Get the forecast start date
    forecast_start = forecast_year['ds'].min()

    # For observed data, only plot history up to the forecast start
    observed = df_city[df_city['ds'] < forecast_start]

    # Decide which columns to use
    fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=False)

    # --- Top subplot: Time series with observed data, Prophet forecast and Hybrid forecast ---
    ax1 = axes[0]
    ax1.set_title(f"Rainfall Forecast for {city} ({target_year})")

    # Plot observed data
    ax1.plot(observed['ds'], observed['y'], label='Observed (History)', color='black')

    # Plot Prophet uncertainty band if available
    if 'yhat_lower' in forecast_year.columns and 'yhat_upper' in forecast_year.columns:
        ax1.fill_between(
            forecast_year['ds'],
            forecast_year['yhat_lower'],
            forecast_year['yhat_upper'],
            color='gray',
            alpha=0.2,
            label='Prophet Uncertainty'
        )

    # Plot Prophet forecast
    ax1.plot(
        forecast_year['ds'], 
        forecast_year['yhat'], 
        label='Prophet Forecast', 
        linestyle='--', 
        color='blue'
    )

    # Plot Hybrid forecast
    ax1.plot(
        forecast_year['ds'], 
        forecast_year['yhat_hybrid'], 
        label='Hybrid Forecast', 
        color='red'
    )

    ax1.set_ylabel("Rainfall (mm)")
    ax1.legend(loc='best')

    # --- Bottom subplot: Monthly average of Hybrid forecast ---
    ax2 = axes[1]
    ax2.set_title(f"Monthly Average Hybrid Forecast ({target_year})")
    forecast_year['month'] = forecast_year['ds'].dt.month
    monthly_avg = forecast_year.groupby('month')['yhat_hybrid'].mean().reset_index().rename(columns={'yhat_hybrid': 'avg_forecast'})
    ax2.bar(monthly_avg['month'], monthly_avg['avg_forecast'])
    ax2.set_xlabel("Month")
    ax2.set_ylabel("Avg Rainfall (mm)")
    ax2.set_xticks(range(1, 13))
    ax2.set_xticklabels([f"{m:02d}" for m in range(1, 13)])
    ax2.set_xlim([0.5, 12.5])

    plt.tight_layout()
    plt.show()

# ------------------------------------------------------------------------
# EXAMPLE USAGE
# ------------------------------------------------------------------------
# Example: Predict for a single city and single year
# (Uncomment and run in your environment)
forecast_for_city = predict_forecast("Kurunegala", 2027)
print(forecast_for_city.head())
plot_full_forecast("Kurunegala", forecast_for_city, 2027)

# Example: Predict for multiple target years for Nuwara Eliya
target_years = [2025, 2026, 2027]
city = "Nuwara Eliya"
for year in target_years:
    forecast_for_city = predict_forecast(city, year)
    print(f"\nForecast for {city} in {year}:")
    print(forecast_for_city.head())
    
    csv_filename = os.path.join(CONFIG["output_dir"], f"{city}_rainfall_forecast_{year}_hybrid.csv")
    forecast_for_city.to_csv(csv_filename, index=False)
    print(f"Forecast saved to {csv_filename}")
    
    plot_full_forecast(city, forecast_for_city, year)
